In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

In [2]:
con=sqlite3.connect('data/im.db')
cursor_obj=con.cursor()
cursor_obj

In [3]:
%%bash
sqlite3 data/im.db
.schema

CREATE TABLE IF NOT EXISTS "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
);
CREATE TABLE IF NOT EXISTS "directors" (
"movie_id" TEXT,
  "person_id" TEXT
);
CREATE TABLE IF NOT EXISTS "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
);
CREATE TABLE IF NOT EXISTS "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
);
CREATE TABLE IF NOT EXISTS "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
);
CREATE TABLE IF NOT EXISTS "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
);
CREATE TABLE IF NOT EXISTS "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
);
CREATE TABLE IF NOT EXISTS "writers" (
"m

In [4]:
q1=\
"""
SELECT
    movie_id,
    primary_title,
    original_title,
    start_year,
    runtime_minutes,
    genres,
    ordering,
    title,
    region,
    language,
    types,
    attributes,
    is_original_title,
    averagerating,
    numvotes

FROM (
    SELECT
        movie_id,
        primary_title,
        original_title,
        start_year,
        runtime_minutes,
        genres,
        ordering,
        title,
        region,
        language,
        types,
        attributes,
        is_original_title
    FROM
        movie_basics
    JOIN
        movie_akas
    USING (movie_id)
        ) as movie_info
JOIN movie_ratings

USING (movie_id)

"""
movie_info=pd.read_sql(q1, con)

In [5]:
movie_info[0:10]

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1,Sangharsh,IN,hi,None,alternative transliteration,0.0,7.0,77
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",2,Sunghursh,None,None,original,None,1.0,7.0,77
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",3,Sunghursh,IN,None,None,None,0.0,7.0,77
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",4,Sunghursh,IN,hi,None,alternative transliteration,0.0,7.0,77
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",5,Sungharsh,IN,hi,None,alternative spelling,0.0,7.0,77
5,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",1,One Day Before the Rainy Season,XWW,en,None,None,0.0,7.2,43
6,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",2,Ashad Ka Ek Din,IN,None,None,None,0.0,7.2,43
7,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",3,One Monsoon Day,None,None,None,None,0.0,7.2,43
8,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",4,Ashad Ka Ek Din,None,None,original,None,1.0,7.2,43
9,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,1,O Outro Lado do Vento,BR,None,imdbDisplay,None,0.0,6.9,4517


In [6]:
q2=\
"""
SELECT 
    movie_id,
    directors.person_id as director_id,
    primary_name as director_primary_name,
    birth_year as director_birth_year,
    death_year as director_death_year,
    primary_profession as director_primary_profession

FROM directors
LEFT JOIN persons 
ON directors.person_id=persons.person_id    
"""
director_info=pd.read_sql(q2, con)

In [7]:
director_info=director_info.drop_duplicates()

In [10]:
movie_w_direct_info=pd.merge(movie_info, director_info, on = 'movie_id', how = 'inner')

In [11]:
movie_w_direct_info=movie_w_direct_info.drop(columns=['ordering','title','region','language','types','attributes','is_original_title','director_birth_year','director_primary_profession']).drop_duplicates().sort_values(by='movie_id')

In [12]:
movie_w_direct_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81206 entries, 0 to 301344
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movie_id               81206 non-null  object 
 1   primary_title          81206 non-null  object 
 2   original_title         81206 non-null  object 
 3   start_year             81206 non-null  int64  
 4   runtime_minutes        73210 non-null  float64
 5   genres                 80486 non-null  object 
 6   averagerating          81206 non-null  float64
 7   numvotes               81206 non-null  int64  
 8   director_id            81206 non-null  object 
 9   director_primary_name  81205 non-null  object 
 10  director_death_year    687 non-null    float64
dtypes: float64(3), int64(2), object(6)
memory usage: 7.4+ MB


In [13]:
movie_w_director_info=movie_w_direct_info[~(movie_w_direct_info['director_death_year']>0)]
movie_w_director_info

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,director_id,director_primary_name,director_death_year
25,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,nm0765384,Valeria Sarmiento,NaN
35,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",4.1,32,nm6883878,Mc Jones,NaN
36,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263,nm0365480,Anthony Harrison,NaN
40,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama,6.8,451,nm1030585,Ágnes Kocsis,NaN
52,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,2013,180.0,Documentary,7.6,53,nm0611850,John Mulholland,NaN
...,...,...,...,...,...,...,...,...,...,...,...
301333,tt9899860,Watching This Movie Is a Crime,Didan in film jorm ast,2019,100.0,"Drama,Thriller",8.1,7,nm10531298,Reza Zehtabchian,NaN
301336,tt9899880,Columbus,Columbus,2018,85.0,Comedy,5.8,5,nm5971240,Hatef Alimardani,NaN
301338,tt9903952,BADMEN with a good behavior,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,nm10532891,Loco Meisenkaiser,NaN
301341,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,nm0151535,T.V. Chandran,NaN


In [14]:
q3=\
"""
SELECT 
    movie_id,
    writers.person_id as writer_id,
    primary_name as writer_primary_name,
    birth_year as writer_birth_year,
    death_year as writer_death_year

FROM writers
LEFT JOIN persons 
ON writers.person_id=persons.person_id    
"""
writer_info=pd.read_sql(q3, con)

In [15]:
writer_info

,movie_id,writer_id,writer_primary_name,writer_birth_year,writer_death_year
0,tt0285252,nm0899854,Tony Vitale,1964.0,NaN
1,tt0438973,nm0175726,Steve Conrad,1968.0,NaN
2,tt0438973,nm1802864,Sean Sorensen,NaN,NaN
3,tt0462036,nm1940585,Bill Haley,NaN,NaN
4,tt0835418,nm0310087,Peter Gaulke,NaN,NaN
...,...,...,...,...,...
255868,tt8999892,nm10122246,Bradley T. Castle,NaN,NaN
255869,tt8999974,nm10122357,Daysi Burbano,NaN,NaN
255870,tt9001390,nm6711477,Bernard Lessa,NaN,NaN
255871,tt9004986,nm4993825,Fredrik Horn Akselsen,NaN,NaN


In [16]:
writer_info=writer_info.drop_duplicates()
writer_info

,movie_id,writer_id,writer_primary_name,writer_birth_year,writer_death_year
0,tt0285252,nm0899854,Tony Vitale,1964.0,NaN
1,tt0438973,nm0175726,Steve Conrad,1968.0,NaN
2,tt0438973,nm1802864,Sean Sorensen,NaN,NaN
3,tt0462036,nm1940585,Bill Haley,NaN,NaN
4,tt0835418,nm0310087,Peter Gaulke,NaN,NaN
...,...,...,...,...,...
255868,tt8999892,nm10122246,Bradley T. Castle,NaN,NaN
255869,tt8999974,nm10122357,Daysi Burbano,NaN,NaN
255870,tt9001390,nm6711477,Bernard Lessa,NaN,NaN
255871,tt9004986,nm4993825,Fredrik Horn Akselsen,NaN,NaN


In [17]:
movie_w_writer_info=pd.merge(movie_info, writer_info, on = 'movie_id', how = 'inner')

In [18]:
movie_w_writer_info=movie_w_writer_info.drop(columns=['ordering','title','region','types','attributes','is_original_title','language','writer_birth_year']).drop_duplicates().sort_values(by='movie_id')

In [19]:
movie_w_writer_info=movie_w_writer_info[~(movie_w_writer_info['writer_death_year']>0)]
movie_w_writer_info

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,writer_id,writer_primary_name,writer_death_year
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,nm0347899,Gulzar,NaN
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,nm1391276,Anjana Rawail,NaN
21,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,nm0462648,Oja Kodar,NaN
46,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,nm0347899,Gulzar,NaN
49,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,nm1360635,Pía Rey,NaN
...,...,...,...,...,...,...,...,...,...,...,...
508672,tt9903952,BADMEN with a good behavior,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,nm10532891,Loco Meisenkaiser,NaN
508675,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,nm0151535,T.V. Chandran,NaN
508679,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,nm10536453,Anooja Kozhikode,NaN
508678,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,nm10536451,Vineesh Aaradya,NaN
